In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torch-geometric pandas numpy scikit-learn networkx matplotlib seaborn tqdm -q

In [ ]:
import time

import numpy as np
import pandas as pd
import psutil
import torch
import torch.nn.functional as F
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv, GCNConv, SAGEConv
from torch_geometric.utils import to_undirected
from ucimlrepo import fetch_ucirepo

try:
    import pynvml
    pynvml.nvmlInit()
    NVML_AVAILABLE = True
except:
    NVML_AVAILABLE = False

Загружаем датасет German Credit.
Содержит 1000 клиентов и 20 признаков (7 числовых и 13 категориальных)
Целевая переменная: "good" (1) или "bad" (2) кредитная история.

In [9]:
german_credit = fetch_ucirepo(id=144)

X = german_credit.data.features
y = german_credit.data.targets 

# Заменяем "good" на 0, "bad" на 1
y = y['class'].values
y = np.where(y == 1, 0, 1)

# Обработка категориальных признаков. pd.get_dummies автоматически кодирует все категориальных колонки. drop_first=True убирает избыточность.
X = pd.get_dummies(X, drop_first=True)

# Стандартизация числовых признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Строим граф методом k-ближайших соседей. Создаётся разреженная матрица смежности с помощью knn. Рёбра только 0/1, петли исключаются. Граф неориентированный.
k = 5
adj_matrix = kneighbors_graph(X_scaled, k, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
edge_index = to_undirected(edge_index)

# train / val / test (60/20/20)
num_nodes = X_scaled.shape[0]
indices = np.arange(num_nodes)

train_idx, temp_idx = train_test_split(indices, test_size=0.4, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

data = Data(
    x=torch.tensor(X_scaled, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(y, dtype=torch.long),
    train_mask=train_mask,
    val_mask=val_mask,
    test_mask=test_mask
)

print(f"Загружено {data.x.shape[0]} узлов, {data.x.shape[1]} признаков")
print(f"Train/Val/Test: {train_mask.sum().item()}/{val_mask.sum().item()}/{test_mask.sum().item()}")
print(f"Рёбер: {edge_index.shape[1]}")

class_counts = torch.bincount(data.y).tolist()
class_dist_str = ", ".join(f"Класс {i}: {count}" for i, count in enumerate(class_counts))
print(f"Распределение классов: {class_dist_str}")

Загружено 1000 узлов, 48 признаков
Train/Val/Test: 600/200/200
Рёбер: 7528
Распределение классов: Класс 0: 700, Класс 1: 300


In [ ]:
# Вычисление весов перед обучением
y_train = data.y[data.train_mask].cpu().numpy()
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(data.x.device)

# Использование в лоссе
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
def compute_metrics(pred, true, probs=None):
    pred = pred.cpu().numpy()
    true = true.cpu().numpy()
    
    metrics = {
        'accuracy': (pred == true).mean(),
        'precision': precision_score(true, pred, zero_division=0),
        'recall': recall_score(true, pred, zero_division=0),
        'f1': f1_score(true, pred, zero_division=0),
    }
    
    if probs is not None:
        probs = probs.cpu().numpy()
        metrics['roc_auc'] = roc_auc_score(true, probs)
        metrics['pr_auc'] = average_precision_score(true, probs)
    
    return metrics

def accuracy(pred_y, y):
    return ((pred_y == y).sum() / len(y)).item()

def test(model, data):
    """Тестирование с полным набором метрик"""
    model.eval()
    with torch.no_grad():
        _, out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)[data.test_mask]
        probs = torch.exp(out)[:, 1][data.test_mask]  # Вероятность класса 1 (дефолт)
        true = data.y[data.test_mask]
        
        return compute_metrics(pred, true, probs)

In [32]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.sage1 = SAGEConv(dim_in, dim_h)
        self.sage2 = SAGEConv(dim_h, dim_out)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                        lr=0.01,
                                        weight_decay=5e-4)

    def forward(self, x, edge_index):
        h = self.sage1(x, edge_index)
        h = torch.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.sage2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        # Взвешенный лосс
        y_train = data.y[data.train_mask].cpu().numpy()
        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        weights = torch.tensor(weights, dtype=torch.float).to(data.x.device)
        criterion = torch.nn.CrossEntropyLoss(weight=weights)
        optimizer = self.optimizer

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            _, out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                train_pred = out[data.train_mask].argmax(dim=1)
                train_metrics = compute_metrics(train_pred, data.y[data.train_mask])
                
                val_pred = out[data.val_mask].argmax(dim=1)
                val_probs = torch.exp(out)[:, 1][data.val_mask]
                val_metrics = compute_metrics(val_pred, data.y[data.val_mask], probs=val_probs)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            if epoch % 10 == 0:
                print(f'Epoch {epoch:>3} | '
                    f'TL: {loss:.3f} | TF1: {train_metrics["f1"]:.3f} | '
                    f'VL: {val_loss:.3f} | VF1: {val_metrics["f1"]:.3f} | '
                    f'VRec: {val_metrics["recall"]:.3f} | VPR: {val_metrics["pr_auc"]:.3f}')

class GAT(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out, heads=4):
        super().__init__()
        self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads, concat=True, dropout=0.6)
        self.gat2 = GATv2Conv(dim_h * heads, dim_out, heads=heads, concat=False, dropout=0.6)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                        lr=0.005,
                                        weight_decay=5e-4)

    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.6, training=self.training)
        h = self.gat1(h, edge_index)
        h = F.elu(h)
        h = F.dropout(h, p=0.6, training=self.training)
        h = self.gat2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        # Взвешенный лосс
        y_train = data.y[data.train_mask].cpu().numpy()
        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        weights = torch.tensor(weights, dtype=torch.float).to(data.x.device)
        criterion = torch.nn.CrossEntropyLoss(weight=weights)
        optimizer = self.optimizer

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            _, out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                train_pred = out[data.train_mask].argmax(dim=1)
                train_metrics = compute_metrics(train_pred, data.y[data.train_mask])
                
                val_pred = out[data.val_mask].argmax(dim=1)
                val_probs = torch.exp(out)[:, 1][data.val_mask]
                val_metrics = compute_metrics(val_pred, data.y[data.val_mask], probs=val_probs)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            if epoch % 10 == 0:
                print(f'Epoch {epoch:>3} | '
                    f'TL: {loss:.3f} | TF1: {train_metrics["f1"]:.3f} | '
                    f'VL: {val_loss:.3f} | VF1: {val_metrics["f1"]:.3f} | '
                    f'VRec: {val_metrics["recall"]:.3f} | VPR: {val_metrics["pr_auc"]:.3f}')

class GCN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_out)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                        lr=0.01,
                                        weight_decay=5e-4)

    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.5, training=self.training)
        h = self.gcn1(h, edge_index)
        h = torch.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.gcn2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        # Взвешенный лосс
        y_train = data.y[data.train_mask].cpu().numpy()
        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        weights = torch.tensor(weights, dtype=torch.float).to(data.x.device)
        criterion = torch.nn.CrossEntropyLoss(weight=weights)
        optimizer = self.optimizer

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            _, out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                train_pred = out[data.train_mask].argmax(dim=1)
                train_metrics = compute_metrics(train_pred, data.y[data.train_mask])
                
                val_pred = out[data.val_mask].argmax(dim=1)
                val_probs = torch.exp(out)[:, 1][data.val_mask]
                val_metrics = compute_metrics(val_pred, data.y[data.val_mask], probs=val_probs)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            if epoch % 10 == 0:
                print(f'Epoch {epoch:>3} | '
                    f'TL: {loss:.3f} | TF1: {train_metrics["f1"]:.3f} | '
                    f'VL: {val_loss:.3f} | VF1: {val_metrics["f1"]:.3f} | '
                    f'VRec: {val_metrics["recall"]:.3f} | VPR: {val_metrics["pr_auc"]:.3f}')

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

def monitor_resources():
    stats = {}
    # CPU & RAM
    stats['ram_mb'] = psutil.virtual_memory().used / (1024 ** 2)
    stats['cpu_percent'] = psutil.cpu_percent()

    # GPU
    if device.type == 'cuda' and NVML_AVAILABLE:
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        util = pynvml.nvmlDeviceGetUtilizationRates(handle)
        stats['gpu_mem_mb'] = mem_info.used / (1024 ** 2)
        stats['gpu_util'] = util.gpu
    else:
        stats['gpu_mem_mb'] = None
        stats['gpu_util'] = None
    return stats

def train_with_monitoring(model, data, epochs, model_name):
    print(f"\n{'='*50}\nTraining {model_name} with resource monitoring\n{'='*50}")
    
    if device.type == 'cuda':
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()

    start_time = time.time()

    model.fit(data, epochs)

    final_ram = psutil.virtual_memory().used / (1024 ** 2)
    max_gpu_mem = None
    if device.type == 'cuda':
        max_gpu_mem = torch.cuda.max_memory_allocated() / (1024 ** 2)

    duration = time.time() - start_time
    test_metrics = test(model, data)

    results = {
        'test_metrics': test_metrics,
        'training_time_sec': duration,
        'final_ram_mb': final_ram,
        'max_gpu_mem_mb': max_gpu_mem,
    }

    print(f"\n{model_name} finished")
    print(f"Test Recall: {test_metrics.get('recall', 0):.3f} | "
          f"F1: {test_metrics.get('f1', 0):.3f} | "
          f"PR-AUC: {test_metrics.get('pr_auc', 0):.3f}")
    print(f"Training Time: {duration:.1f} sec")
    if max_gpu_mem:
        print(f"Peak GPU Memory: {max_gpu_mem:.1f} MB")
    print(f"Final RAM Usage: {final_ram:.1f} MB")

    return test_metrics.get('recall', 0), results

In [34]:
%%time

results = {}

# 1. GraphSAGE
graphsage = GraphSAGE(data.x.shape[1], dim_h=64, dim_out=2).to(device)
acc_sage, _ = train_with_monitoring(graphsage, data, epochs=100, model_name="GraphSAGE")
results['GraphSAGE'] = acc_sage


Training GraphSAGE with resource monitoring
Epoch   0 | TL: 0.728 | TF1: 0.368 | VL: 0.722 | VF1: 0.446 | VRec: 0.524 | VPR: 0.378
Epoch  10 | TL: 0.471 | TF1: 0.690 | VL: 0.586 | VF1: 0.571 | VRec: 0.635 | VPR: 0.616
Epoch  20 | TL: 0.362 | TF1: 0.779 | VL: 0.640 | VF1: 0.584 | VRec: 0.635 | VPR: 0.621
Epoch  30 | TL: 0.292 | TF1: 0.828 | VL: 0.739 | VF1: 0.555 | VRec: 0.603 | VPR: 0.630
Epoch  40 | TL: 0.251 | TF1: 0.825 | VL: 0.951 | VF1: 0.525 | VRec: 0.508 | VPR: 0.614
Epoch  50 | TL: 0.214 | TF1: 0.859 | VL: 0.836 | VF1: 0.590 | VRec: 0.651 | VPR: 0.691
Epoch  60 | TL: 0.153 | TF1: 0.901 | VL: 1.185 | VF1: 0.585 | VRec: 0.603 | VPR: 0.587
Epoch  70 | TL: 0.147 | TF1: 0.904 | VL: 1.203 | VF1: 0.515 | VRec: 0.540 | VPR: 0.600
Epoch  80 | TL: 0.110 | TF1: 0.940 | VL: 1.543 | VF1: 0.512 | VRec: 0.492 | VPR: 0.564
Epoch  90 | TL: 0.103 | TF1: 0.952 | VL: 1.557 | VF1: 0.525 | VRec: 0.508 | VPR: 0.584
Epoch 100 | TL: 0.078 | TF1: 0.955 | VL: 1.777 | VF1: 0.508 | VRec: 0.524 | VPR: 0.51

In [35]:
%%time

# 2. GCN
gcn = GCN(data.x.shape[1], dim_h=64, dim_out=2).to(device)
acc_gcn, _ = train_with_monitoring(gcn, data, epochs=100, model_name="GCN")
results['GCN'] = acc_gcn


Training GCN with resource monitoring
Epoch   0 | TL: 0.769 | TF1: 0.404 | VL: 0.744 | VF1: 0.408 | VRec: 0.683 | VPR: 0.348
Epoch  10 | TL: 0.581 | TF1: 0.583 | VL: 0.634 | VF1: 0.553 | VRec: 0.619 | VPR: 0.527
Epoch  20 | TL: 0.548 | TF1: 0.634 | VL: 0.616 | VF1: 0.567 | VRec: 0.635 | VPR: 0.562
Epoch  30 | TL: 0.557 | TF1: 0.588 | VL: 0.610 | VF1: 0.548 | VRec: 0.635 | VPR: 0.542
Epoch  40 | TL: 0.531 | TF1: 0.627 | VL: 0.657 | VF1: 0.521 | VRec: 0.603 | VPR: 0.491
Epoch  50 | TL: 0.514 | TF1: 0.625 | VL: 0.661 | VF1: 0.538 | VRec: 0.667 | VPR: 0.519
Epoch  60 | TL: 0.510 | TF1: 0.638 | VL: 0.719 | VF1: 0.550 | VRec: 0.571 | VPR: 0.519
Epoch  70 | TL: 0.504 | TF1: 0.640 | VL: 0.700 | VF1: 0.561 | VRec: 0.698 | VPR: 0.467
Epoch  80 | TL: 0.500 | TF1: 0.630 | VL: 0.682 | VF1: 0.518 | VRec: 0.571 | VPR: 0.503
Epoch  90 | TL: 0.483 | TF1: 0.645 | VL: 0.744 | VF1: 0.517 | VRec: 0.619 | VPR: 0.500
Epoch 100 | TL: 0.492 | TF1: 0.643 | VL: 0.645 | VF1: 0.562 | VRec: 0.683 | VPR: 0.569

GCN

In [36]:
%%time

# 3. GAT
gat = GAT(data.x.shape[1], dim_h=32, dim_out=2, heads=4).to(device)
acc_gat, _ = train_with_monitoring(gat, data, epochs=100, model_name="GAT")
results['GAT'] = acc_gat


Training GAT with resource monitoring


Epoch   0 | TL: 0.974 | TF1: 0.298 | VL: 0.871 | VF1: 0.239 | VRec: 0.206 | VPR: 0.368
Epoch  10 | TL: 0.676 | TF1: 0.508 | VL: 0.663 | VF1: 0.488 | VRec: 0.667 | VPR: 0.469
Epoch  20 | TL: 0.611 | TF1: 0.519 | VL: 0.678 | VF1: 0.500 | VRec: 0.603 | VPR: 0.467
Epoch  30 | TL: 0.629 | TF1: 0.533 | VL: 0.631 | VF1: 0.525 | VRec: 0.667 | VPR: 0.466
Epoch  40 | TL: 0.611 | TF1: 0.586 | VL: 0.631 | VF1: 0.583 | VRec: 0.778 | VPR: 0.485
Epoch  50 | TL: 0.629 | TF1: 0.541 | VL: 0.611 | VF1: 0.595 | VRec: 0.746 | VPR: 0.547
Epoch  60 | TL: 0.596 | TF1: 0.540 | VL: 0.629 | VF1: 0.528 | VRec: 0.667 | VPR: 0.506
Epoch  70 | TL: 0.585 | TF1: 0.573 | VL: 0.640 | VF1: 0.575 | VRec: 0.698 | VPR: 0.562
Epoch  80 | TL: 0.604 | TF1: 0.593 | VL: 0.628 | VF1: 0.553 | VRec: 0.667 | VPR: 0.558
Epoch  90 | TL: 0.591 | TF1: 0.571 | VL: 0.662 | VF1: 0.510 | VRec: 0.587 | VPR: 0.464
Epoch 100 | TL: 0.619 | TF1: 0.534 | VL: 0.646 | VF1: 0.500 | VRec: 0.603 | VPR: 0.468

GAT finished
Test Recall: 0.691 | F1: 0.53